In [26]:
import os
import gensim.models as models
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import random,numpy as np

def read_total_data(): # reading the total data
    data = {}
    for i in os.listdir('20_newsgroups/'):
        corpus = []
        for j in sorted(os.listdir('20_newsgroups/'+i)):
            temp_data = open('20_newsgroups/'+i+'/'+j,'rb').read().decode('utf-8', 'ignore').lower()
            temp_data = word_tokenize(temp_data)
            corpus.append(temp_data)
        data[i] = corpus
    return data

def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [18]:
import os
import gensim.models as models
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import random,numpy as np
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
# from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import operator, random
import sys
import math
import pickle
from nltk.stem import *
from num2words import num2words
from wordcloud import STOPWORDS

def remove_header_footer(final_string):
	new_final_string=""
	flag=1
	tokens=final_string.split('\n\n')
	# Remove tokens[0] and tokens[-1]
	for token in tokens[1:-1]:
		flag+=1
		new_final_string+=str(token)+" "
	flag=0
	return new_final_string

def remove_html(data):
	return BeautifulSoup(data, "html.parser").get_text()

# def remove_btw_sqr(data):
#     fin = re.sub('\[[^]]*\]', '', data)
#     return fin

def fix_contractions(data):
    fin = contractions.fix(data)
    return fin

def words_tokenizer(data):
	words = nltk.word_tokenize(data)
	# tknzr = TweetTokenizer()	
	# tknzr.tokenize(data)
	return words

def remove_non_ascii(words):
	new_words = []
	flag = 0
	for i in range(len(words)):
		flag = 1
		new_word = unicodedata.normalize('NFKD',unicode(words[i]))
		new_word = new_word.encode('ascii','ignore')
		new_word = new_word.decode('utf-8','ignore')
		flag+=1
		new_words.append(new_word)
	return new_words
# def remove_non_ascii(words):
#     new_words = []
#     flag = 0
#     for i in range(len(words)):
#  		flag=1
#  		new_word = unicodedata.normalize('NFKD', words[i]).encode('ascii', 'ignore').decode('utf-8', 'ignore')
#  		flag+=1
#  		new_words.append(new_word)
#  	return new_words

def to_lowercase(words):
    new_words = []
    flag = 0
    for i in range(len(words)):
        new_word = words[i].lower()
        flag+=1
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    new_words = []
    flag = 0
    for i in range(len(words)):
    	flag+=1
        new_word = re.sub(r'([^\w\s])|_+', '', words[i])
        if new_word != '':
        	flag=0
        	new_words.append(new_word)
    return new_words

# def replace_numbers(words):
#     p = inflect.engine()
#     new_words = []
#     flag = 0
#     for i in range(len(words)):
#     	flag = 1
#         if words[i].isdigit():
#             new_word = p.number_to_words(words[i])
#             flag+=1
#             new_words.append(new_word)
#         else:
#         	flag = 0
#         	new_words.append(words[i])
#     return new_words

def replace_numbers(words):
	new_words = []
	for i in range(len(words)):
		if words[i].isdigit():
			temp_word = num2words(words[i])
			new_words.append(temp_word)
		else:
			new_words.append(words[i])
	return new_words

def remove_stopwords(words):
    new_words = []
    flag = 0
    for i in range(len(words)):
    	flag = 1
        if words[i] not in stopwords.words('english') and words[i] not in STOPWORDS:
        	flag+=1
        	new_words.append(words[i])
    return new_words

def remove_numbers(words):
    new_words = []
    flag=0
    for i in range(len(words)):
        if words[i].isdigit():
            flag+=1
        else:
            new_words.append(words[i])
    return new_words

def stemming(words):
	new_words = []
	stemmer = PorterStemmer()
	for i in range(len(words)):
		new_words.append(stemmer.stem(words[i]))
	return new_words

def preprocess_input_sentence(data):
	# data = remove_header_footer(data)
	data = remove_html(data)
	# data = remove_btw_sqr(data)
	data = fix_contractions(data)
	words = words_tokenizer(data)
	words = remove_non_ascii(words)
	words = to_lowercase(words)
	words = remove_punctuation(words)
	words = replace_numbers(words)
# 	words = stemming(words)
	words = remove_stopwords(words)
	return words

file_mapping_count = -1
prepro_data_dic = {}
count_to_name = {}
name_to_count = {}
file_titles = {}
toremove = []
ground_labels = []

with open('prepro_files.pkl') as f:
	prepro_data_dic = pickle.load(f)

with open('name_to_count.pkl') as f:
	name_to_count = pickle.load(f)

with open('count_to_name.pkl') as f:
	count_to_name = pickle.load(f)

with open('ground_labels.pkl') as f:
	ground_labels = pickle.load(f)

# ground_count = -1
# for i in os.listdir('20_newsgroups/'):
#     ground_count+=1
#     for j in sorted(os.listdir('20_newsgroups/'+i)):
#         file_mapping_count+=1
# #         print file_mapping_count
#         file_name = i+'/'+j
# #         print file_name
#         count_to_name[file_mapping_count] = file_name
#         name_to_count[file_name] = file_mapping_count
#         file_name_path = '20_newsgroups/'+i+'/'+j
#         temp_data = open(file_name_path,'rb').read().decode('utf-8', 'ignore').lower()
#         prepro_data = preprocess_input_sentence(temp_data)
#         prepro_data_dic[file_mapping_count] = temp_data
#         ground_labels.append(ground_count)


In [20]:
import random

def data_splitting(data,labels,ratio,seed):
    
    classwise_data = {}
    for i in range(len(labels)):
        if labels[i] not in classwise_data:
            classwise_data[labels[i]] = []
        classwise_data[labels[i]].append(data[i])
    
    final_train_data = []
    final_test_data = []
    final_train_labels = []
    final_test_labels = []
    final_train_docids = []
    final_test_docids = []
    
    for k,v in classwise_data.iteritems():
        shuf_ind = []
        for i in range(len(v)):
            shuf_ind.append(i)
        random.Random(seed).shuffle(shuf_ind)
        
        lim = int(len(shuf_ind)*ratio)
        train_ind = shuf_ind[:lim]
        test_ind = shuf_ind[lim:]

        train_data = []
        train_labels = []
        test_data = []
        test_labelsabels = []
        for i in range(len(train_ind)):
            train_data.append(classwise_data[k][train_ind[i]])
            train_labels.append(k)
        for i in range(len(test_ind)):
            test_data.append(classwise_data[k][test_ind[i]])
            test_labels.append(k)
        final_train_data+=train_data
        final_test_data+=test_data
        final_train_labels+=train_labels
        final_test_labels+=test_labels
        
    return final_train_data,final_train_labels,final_test_data,final_test_labels


In [22]:
train_data,train_labels,test_data,test_labels = data_splitting(prepro_data_dic,ground_labels,0.5,6)

In [28]:
import gensim

train_tagged_data = []
for i in range(len(train_data)):
    train_tagged_data.append(gensim.models.doc2vec.TaggedDocument(train_data[i], [i]))

In [29]:
train_tagged_data[:2]

[TaggedDocument(words=[u'path', u'cantaloupesrvcscmuedu', u'magnesiumclubcccmuedu', u'newsseicmuedu', u'cisohiostateedu', u'pacificmpsohiostateedu', u'zaphodmpsohiostateedu', u'howlandrestonansnet', u'gatech', u'ukma', u'netnewslouisvilleedu', u'wkuvx1', u'kozloc', u'kozloc', u'wkuvx1bitnet', u'newsgroup', u'recsporthockey', u'subject', u'tie', u'breaker', u'isl', u'devil', u'dir', u'messageid', u'1993apr201659416920', u'wkuvx1bitnet', u'date', u'twenty point zero', u'apr', u'ninety-three point zero', u'one hundred and sixty-five thousand, nine hundred and forty-one point zero', u'cdt', u'refer', u'1993apr182221156525', u'ramseycslaurentianca', u'organ', u'western', u'kentucki', u'univers', u'bowl', u'green', u'ky', u'line', u'twenty-five point zero', u'articl', u'1993apr182221156525', u'ramseycslaurentianca', u'maynard', u'ramseycslaurentianca', u'roger', u'maynard', u'write', u'wangr', u'vccsouth22itsrpiedu', u'rex', u'wang', u'write', u'might', u'great', u'math', u'tell', u'two', u'

In [30]:
test_data[:2]

[[u'path',
  u'cantaloupesrvcscmuedu',
  u'crabapplesrvcscmuedu',
  u'fs7ececmuedu',
  u'europaenggtefsdcom',
  u'howlandrestonansnet',
  u'nocnearnet',
  u'ctronnewsctroncom',
  u'ctroncom',
  u'machnik',
  u'machnik',
  u'ctroncom',
  u'michael',
  u'c',
  u'machnik',
  u'newsgroup',
  u'recsporthockey',
  u'subject',
  u'goali',
  u'stat',
  u'date',
  u'twenty-one point zero',
  u'apr',
  u'one thousand, nine hundred and ninety-three point zero',
  u'two hundred and thirteen thousand, four hundred and thirteen point zero',
  u'gmt',
  u'organ',
  u'cabletron',
  u'system',
  u'inc',
  u'line',
  u'one hundred and twenty-four point zero',
  u'messageid',
  u'1r4eklinnl2l',
  u'ctronnewsctroncom',
  u'nntppostinghost',
  u'pongctroncom',
  u'nin15b34',
  u'merrimackedu',
  u'articl',
  u'1r3smf',
  u'67m',
  u'accessusaskca',
  u'coulman',
  u'csusaskca',
  u'randi',
  u'coulman',
  u'write',
  u'path',
  u'ctronnewsctroncom',
  u'nocnearnet',
  u'uunet',
  u'utcsri',
  u'newsflashco

In [31]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)


In [33]:
model.build_vocab(train_tagged_data)


In [35]:
%time model.train(train_tagged_data, total_examples=model.corpus_count, epochs=model.epochs)


CPU times: user 1min 3s, sys: 994 ms, total: 1min 4s
Wall time: 25.8 s


In [38]:
train_vectors = []
for i in range(len(train_data)):
    tvec = model.infer_vector(train_data[i])
    train_vectors.append(tvec)

test_vectors = []
for i in range(len(test_data)):
    tvec = model.infer_vector(test_data[i])
    test_vectors.append(tvec)


In [39]:
# with open('train_vectors.pkl','wb') as f:
#     f.write(pickle.dumps(train_vectors))

# with open('test_vectors.pkl','wb') as f:
#     f.write(pickle.dumps(test_vectors))

In [57]:
predictions = []
groundv = []
for i in range(len(test_vectors)):
    sim_scores = {}
    for j in range(len(train_vectors)):
        t_num = np.dot(train_vectors[j], test_vectors[i])
        t_deno = np.linalg.norm(train_vectors[j])*np.linalg.norm(test_vectors[i])
        sim_scores[j] = ((t_num)/((t_deno)*1.0))
    sorted_sim_results = sorted(sim_scores.items(), key=operator.itemgetter(1),reverse=True)
    predictions.append(train_labels[sorted_sim_results[:1][0][0]]) 
    groundv.append(test_labels[i])
    


In [65]:
cor_count = 0
for i in range(len(predictions)):
    if predictions[i] == groundv[i]:
        cor_count+=1

print("Accuracy for k=1: " + str(((cor_count*1.0)/len(test_data))*100) + "%")
print ""

import pandas as pd 

mat = np.zeros((5, 5))
flag=0
for it in range(len(predictions)):
    flag+=1
    mat[int(test_labels[it])][int(predictions[it])] += 1

df = pd.DataFrame(mat)
print("True vs Predicted")
flag+=1
print(mat)
print("")

Accuracy for k=1: 90.76%

True vs Predicted
[[488.   2.   1.   5.   4.]
 [  7. 441.  10.  25.  17.]
 [ 14.  10. 432.  26.  18.]
 [  9.  23.  13. 449.   6.]
 [ 11.   9.  17.   4. 459.]]



In [66]:
# with open('predictions_k1_50.pkl','wb') as f:
#     f.write(pickle.dumps(predictions))

# with open('ground_k1_50.pkl','wb') as f:
#     f.write(pickle.dumps(groundv))

In [78]:
predictions = []
groundv = []
for i in range(len(test_vectors)):
    sim_scores = {}
    for j in range(len(train_vectors)):
        t_num = np.dot(train_vectors[j], test_vectors[i])
        t_deno = np.linalg.norm(train_vectors[j])*np.linalg.norm(test_vectors[i])
        sim_scores[j] = ((t_num)/((t_deno)*1.0))
    sorted_sim_results = sorted(sim_scores.items(), key=operator.itemgetter(1),reverse=True)
    t3 = sorted_sim_results[:5]
    tlabels = []
    for j in range(len(t3)):
        tlb = t3[j][0]
        tlabels.append(train_labels[tlb])
    max_label = {}
    for j in range(len(tlabels)):
        if tlabels[j] not in max_label:
            max_label[tlabels[j]] = 0
        max_label[tlabels[j]] += 1
    sorted_labels = sorted(sim_scores.items(), key=operator.itemgetter(1),reverse=True)
    predictions.append(train_labels[sorted_labels[:1][0][0]]) 
    groundv.append(test_labels[i])
    


In [79]:
cor_count = 0
for i in range(len(predictions)):
    if predictions[i] == groundv[i]:
        cor_count+=1

print("Accuracy for k=3: " + str(((cor_count*1.0)/len(test_data))*100) + "%")
print ""

import pandas as pd 

mat = np.zeros((5, 5))
flag=0
for it in range(len(predictions)):
    flag+=1
    mat[int(test_labels[it])][int(predictions[it])] += 1

df = pd.DataFrame(mat)
print("True vs Predicted")
flag+=1
print(mat)
print("")

Accuracy for k=3: 90.76%

True vs Predicted
[[488.   2.   1.   5.   4.]
 [  7. 441.  10.  25.  17.]
 [ 14.  10. 432.  26.  18.]
 [  9.  23.  13. 449.   6.]
 [ 11.   9.  17.   4. 459.]]



In [86]:
def roc_curve(predicted,actual):
    tp = 0
    fp = 0
    for i,j in zip(predicted,actual):
        print i,j
        if i==1:
            if j==1:
                tp+=1
            else:
                fp+=1
    return tp,fp

In [87]:
tp,fp = roc_curve(predictions,groundv)

0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
4 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
2 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
4 0
0 0
0 0
0 0


In [83]:
print tp,fp

441 44
